In [ ]:
import os

import numpy
import pandas
import yaml
from tqdm import tqdm

In [ ]:
os.listdir("./data/toktarova_et_al_2019/")

In [ ]:
with open("./data/gegis__all_countries.yaml", "r") as file:
    data = yaml.safe_load(file)

In [ ]:
items = data["items"]
gegis_countries = [[item["country_name"], item["country_code"]] for item in items]

In [ ]:
gegis_country_codes = numpy.array(gegis_countries).T[1]

In [ ]:
gegis_countries

## Features

"""

We take time series of hourly electricity demand for 44 countries from Toktarova et al. [15] and fit a gradient boosting regression model [32] to demand time series for each country normalized to their annual mean

Estimates of annual country-level annual electricity generation in 2050 were produced by extrapolating annual demand in 2016 [33] using regional demand growth in the SSP2-26 scenario

Therefore we chose to train our model on ten independent var­
iables: 

(i + ii) annual per-capita electricity demand and purchase-power adjusted GDP (for prediction, we extrapolated this to 2050 using the SSP2 scenario in a similar way to demand as above), 

(iii) average hourly temperature profiles over the year in the 3 most densely populated areas of each country [35,36], 

(iv) the mean annual temperature level,

(v) the 1st temperature percentile across the year (to represent how low the temperature dips go),

(vi) the 99th percentile (to represent how high temperature spikes go),

(vii) hour of the day,

(viii) a weekday/weekend indicator,

(ix) mean monthly temperature levels, and

(x) a temperature-based ranking of months of the year
(where the first month is the coldest month, and the month ranked last is the warmest across the year).
The temperature ranking of months was chosen in order to reflect that different countries have summer in different calendar months.

"""



### (i) annual per-capita electricity demand

In [ ]:
ElectricityPerCapita = pandas.read_csv(
    "./data/toktarova_et_al_2019/ElectricityperCapita.csv", index_col=0, header=1
)

In [ ]:
ElectricityPerCapita.columns = [
    int(float(col_name)) if col_name.split(".")[0].isdigit() else col_name
    for col_name in ElectricityPerCapita.columns
]

In [ ]:
ElectricityPerCapita.head()

In [ ]:
import pycountry


def search_pycountry(country_name: str) -> str | None:
    try:
        # Try to find the country
        country = pycountry.countries.search_fuzzy(country_name)[0]
        return country.alpha_2
    except LookupError:
        return None


def get_country_codes(country_names):
    country_codes = []
    for country_name in country_names:
        found_country_code = search_pycountry(country_name)
        if not (found_country_code):
            print("Not Found:", country_name)

        country_codes.append(found_country_code)
    return country_codes

In [ ]:
ElectricityPerCapita.insert(
    1, "country_code", get_country_codes(ElectricityPerCapita["Countries"])
)

In [ ]:
# Print gegis countries not found by search
for code in gegis_country_codes:
    if code not in ElectricityPerCapita["country_code"].values:
        print(code)

In [ ]:
# Adjust the missing countries
index_cc_BA = ElectricityPerCapita[
    ElectricityPerCapita["Countries"] == "Bosnia-Herzegovina"
].index
ElectricityPerCapita.loc[index_cc_BA, "country_code"] = "BA"

index_cc_RS = ElectricityPerCapita[
    ElectricityPerCapita["Countries"] == "Serbia (former Yugoslavia)"
].index
ElectricityPerCapita.loc[index_cc_RS, "country_code"] = "RS"

index_cc_KR = ElectricityPerCapita[
    ElectricityPerCapita["Countries"] == "Korea. Republic of"
].index
ElectricityPerCapita.loc[index_cc_KR, "country_code"] = "KR"

index_cc_TR = ElectricityPerCapita[ElectricityPerCapita["Countries"] == "Turkey"].index
ElectricityPerCapita.loc[index_cc_TR, "country_code"] = "TR"

In [ ]:
list_ids_gegis = []
for i in range(len(ElectricityPerCapita)):
    current_row = ElectricityPerCapita.iloc[i]

    if current_row["country_code"] in gegis_country_codes:
        list_ids_gegis += [i]

In [ ]:
ElectricityPerCapita = ElectricityPerCapita.iloc[list_ids_gegis]

In [ ]:
# Drop manually found incorrect countries (usually territories)
ElectricityPerCapita = ElectricityPerCapita.drop([207, 211, 220])

In [ ]:
ElectricityPerCapita[["country_code", 2015]]

In [ ]:
toktarova_indicies = ElectricityPerCapita.index

In [ ]:
ElectricityPerCapita_2015 = ElectricityPerCapita[["country_code", 2015]].copy()
ElectricityPerCapita_2015.rename(
    columns={2015: "ElectricityPerCapita_2015"}, inplace=True
)

In [ ]:
ElectricityPerCapita_2015.head()

## (ii) purchase-power adjusted GDP 

In [ ]:
GDP_PPP = pandas.read_csv(
    "./data/toktarova_et_al_2019/GDPperCapita.csv", index_col=0, header=1
)

In [ ]:
GDP_PPP.columns = [
    int(float(col_name)) if col_name.split(".")[0].isdigit() else col_name
    for col_name in GDP_PPP.columns
]

In [ ]:
GDP_PPP[2015].count()

In [ ]:
GDP_PPP = GDP_PPP.loc[toktarova_indicies]

In [ ]:
GDP_PPP.insert(1, "country_code", ElectricityPerCapita["country_code"])

In [ ]:
GDP_PPP.head()

In [ ]:
GDP_2015 = GDP_PPP[["country_code", 2015]].copy()
GDP_2015.rename(columns={2015: "GDP_PPP_2015"}, inplace=True)

In [ ]:
GDP_2015.head()

### (iii) average hourly temperature profiles over the year in the 3 most densely populated areas of each country [35,36]

In [ ]:
from dotenv import dotenv_values

config = dotenv_values(".env")

In [ ]:
# ETL/temperature
storage_bucket = config["GCSBUCKET"]

In [ ]:
df_temperature_top_3 = pandas.DataFrame()
for country_code in tqdm(gegis_country_codes, desc="Processing countries"):  # +
    current_df = pandas.read_csv(
        f"{storage_bucket}/temperature/temperature_time_series_top_3_{country_code}_2015.csv"
    )
    current_df.reset_index(inplace=True)
    current_df["country_code"] = country_code
    df_temperature_top_3 = pandas.concat(
        [df_temperature_top_3, current_df], ignore_index=True
    )

In [ ]:
df_temperature_top_3.rename(
    columns={"Temperature (K)": "Temperature top_3 (K)"}, inplace=True
)

In [ ]:
df_temperature_top_3.head()

In [ ]:
df_temperature_top_3.to_parquet("./data/temperature_top_3.parquet")

In [ ]:
df_temperature_top_3 = pandas.read_parquet("./data/temperature_top_3.parquet")

## ETL/temperature

(iv) the mean annual temperature level,

(v) the 1st temperature percentile across the year (to represent how low the temperature dips go), (5th percentile in our case)

(vi) the 99th percentile (to represent how high temperature spikes go), (95th percentile in our case)

(vii) hour of the day,

(viii) a weekday/weekend indicator,

(ix) mean monthly temperature levels, and

(x) a temperature-based ranking of months of the year

In [ ]:
df_temperature_top_1 = pandas.DataFrame()
for country_code in tqdm(gegis_country_codes, desc="Processing countries"):  # +
    current_df = pandas.read_csv(
        f"{storage_bucket}/temperature/temperature_time_series_top_1_{country_code}_2015.csv"
    )
    current_df.reset_index(inplace=True)
    current_df["country_code"] = country_code
    df_temperature_top_1 = pandas.concat(
        [df_temperature_top_1, current_df], ignore_index=True
    )

In [ ]:
df_temperature_top_1["weekend_indicator"] = (
    df_temperature_top_1["Local day of the week"] >= 5
).astype(int)

In [ ]:
df_temperature_top_1.head()

In [ ]:
df_temperature_top_1.to_parquet("./data/temperature_top_1.parquet")

In [ ]:
df_temperature_top_1 = pandas.read_parquet("./data/temperature_top_1.parquet")

## Target hourly electricity load data

In [ ]:
Hourly_demand = pandas.read_csv(
    "./data/toktarova_et_al_2019/Real load hourly data.csv", index_col=0, header=0
)

In [ ]:
Hourly_demand = Hourly_demand.T

In [ ]:
Hourly_demand.index = Hourly_demand.index.astype(int)

In [ ]:
selected_hourly_demand = Hourly_demand.loc[toktarova_indicies]

In [ ]:
selected_hourly_demand.insert(1, "country_code", ElectricityPerCapita["country_code"])

In [ ]:
temp = selected_hourly_demand.loc[9]

In [ ]:
selected_hourly_demand.head()

In [ ]:
selected_hourly_demand.drop(
    columns=[
        "Electricity consumption",
        "Countryname",
        "annual electricity consumption in TWh",
        "average",
        "R",
        "Psyn",
        "Preal",
    ],
    inplace=True,
)

In [ ]:
selected_hourly_demand.columns

In [ ]:
selected_hourly_demand.columns = [
    int(col_name.split(" ")[0].split("_")[-1]) - 1
    if col_name.split(" ")[0].split("_")[-1].isdigit()
    else col_name
    for col_name in selected_hourly_demand.columns
]

In [ ]:
selected_hourly_demand.head()

In [ ]:
all_hourly_demand = pandas.DataFrame()

for i in tqdm(range(len(selected_hourly_demand)), desc="Processing countries"):
    current_country_hourly_demand = pandas.DataFrame(selected_hourly_demand.iloc[i, 1:])
    current_country_hourly_demand.columns = ["hourly_demand"]

    current_country_hourly_demand["country_code"] = selected_hourly_demand.iloc[i, 0]
    current_country_hourly_demand.reset_index(inplace=True)

    all_hourly_demand = pandas.concat(
        [all_hourly_demand, current_country_hourly_demand], ignore_index=True
    )

In [ ]:
all_hourly_demand["country_code"].unique()

## Combine into a single dataframe for training

In [ ]:
combined_dataset = pandas.merge(
    all_hourly_demand, df_temperature_top_1, on=["country_code", "index"]
)

In [ ]:
combined_dataset = pandas.merge(
    combined_dataset,
    df_temperature_top_3[["index", "country_code", "Temperature top_3 (K)"]],
    on=["country_code", "index"],
)

In [ ]:
combined_dataset = pandas.merge(combined_dataset, GDP_2015, on=["country_code"])
combined_dataset = pandas.merge(
    combined_dataset, ElectricityPerCapita_2015, on=["country_code"]
)

In [ ]:
combined_dataset.columns

In [ ]:
combined_dataset.drop(
    columns=[
        "index",
        "country_code",
        "Time (UTC)",
        "Local day of the week",
        "Local year",
        "Annual average temperature (K)",
    ],
    inplace=True,
)
# To get the same as GEGIS: https://github.com/niclasmattsson/GlobalEnergyGIS?tab=readme-ov-file#synthetic-demand-list-of-training-variables

In [ ]:
combined_dataset

### Model

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [ ]:
# Features
features = combined_dataset.drop(columns=["hourly_demand"])

In [ ]:
features.head()

In [ ]:
target = combined_dataset["hourly_demand"]
target.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.2, random_state=42
)

In [ ]:
# Initialize the XGBoost regressor
xgb_model = XGBRegressor(
    n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42
)

# Train the model
xgb_model.fit(X_train, y_train)

In [ ]:
# Make predictions
y_pred = xgb_model.predict(X_test)

# Calculate MSE and R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

In [ ]:
import matplotlib.pyplot as plt

# Get feature importance
importance = xgb_model.feature_importances_
feature_names = features.columns

# Sort features by importance
indices = numpy.argsort(importance)

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.title("Feature Importance")
plt.barh(range(len(importance)), importance[indices])
plt.yticks(range(len(importance)), [feature_names[i] for i in indices])
plt.xlabel("Importance")
plt.tight_layout()
plt.show()